In [1]:
import numpy as np #Importation de numpy
import pandas as pd #Importation de pandas
import nltk #Importation de nltk
import sklearn #Importation de sklearn
from nltk.corpus import stopwords #Importation des stopwords
import string #Import de string pour la fonction de trie

In [2]:
df = pd.read_csv("input/emails.csv") #Permets de mettre les données de la base de données emails dans df

In [3]:
df.shape #Renvoi la taille du tableau df

(5728, 2)

In [4]:
df.head(5) #Affiche les 5 premières lignes du tableau df

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [5]:
df.columns #Affiche le nom des colonnes et le type du tableau

Index(['text', 'spam'], dtype='object')

In [6]:
df.drop_duplicates(inplace = True) #Supprime tous les doublons

In [7]:
df.shape #Réaffiche la taille du tableau après suppression des doublons

(5695, 2)

In [8]:
df.isnull().sum() #Permets de voir s'il y a des lignes avec rien dedans

text    0
spam    0
dtype: int64

In [9]:
nltk.download('stopwords') #Télécharge la base de données des stopwords de python (Stropwords = mots inutiles)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexandre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
def mot_trie(text): #Fonction pour trier les mots (enlever les ponctuations et les stopwords)
    nopunc = [char for char in text if char not in string.punctuation] #Permets d'enlever les ponctuations
    nopunc = ''.join(nopunc)
    
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')] #Enlève tous les stopwords anglais car base de données anglaise 
    
    return clean_words #Retourne la liste des mots triés
    

In [11]:
df['text'].head().apply(mot_trie) #Affiche la fonction mot_trie sur toutes les lignes de la colonne text du tableau df (affichage possible grâce à l'import de string)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

In [12]:
from sklearn.feature_extraction.text import CountVectorizer #Importation de CountVectorizer
matrice_spam = CountVectorizer(analyzer=mot_trie).fit_transform(df['text']) 
#Converti le texte trié grâce à la fonction mot-trie de la colonne text en une matrice de jetons, chaque mot unique va être dans une matrice avec leur nombre d'apparition  

In [13]:
print(matrice_spam) 
#On peut voir que sur la première colonne (0,3638), avec le 0 pour dire qu'il sagit de la première ligne de la colonne text, le 3638 est le numéro du mot et sur la seconde colonne il s'agit de son nombre d'apparition

  (0, 3638)	1
  (0, 23369)	1
  (0, 18841)	1
  (0, 10065)	1
  (0, 17696)	1
  (0, 21140)	1
  (0, 27986)	1
  (0, 16674)	1
  (0, 28110)	1
  (0, 9296)	3
  (0, 21654)	2
  (0, 15429)	1
  (0, 32602)	1
  (0, 18238)	1
  (0, 18886)	1
  (0, 16089)	2
  (0, 8054)	1
  (0, 20952)	3
  (0, 32319)	1
  (0, 31968)	1
  (0, 24838)	1
  (0, 36025)	2
  (0, 21431)	2
  (0, 33037)	1
  (0, 23040)	2
  :	:
  (5694, 24818)	2
  (5694, 21624)	1
  (5694, 5729)	9
  (5694, 30934)	1
  (5694, 2828)	3
  (5694, 13338)	1
  (5694, 13127)	1
  (5694, 17388)	1
  (5694, 14130)	1
  (5694, 20273)	1
  (5694, 31827)	1
  (5694, 13128)	1
  (5694, 20467)	1
  (5694, 35288)	1
  (5694, 8629)	1
  (5694, 30082)	1
  (5694, 13522)	5
  (5694, 36185)	1
  (5694, 959)	2
  (5694, 2797)	1
  (5694, 30287)	1
  (5694, 17590)	1
  (5694, 33923)	1
  (5694, 10373)	1
  (5694, 11386)	1


In [14]:
from sklearn.model_selection import train_test_split #Importation de train_test_split
X_train, X_test, y_train, y_test = train_test_split(matrice_spam, df['spam'], test_size = 0.30, random_state = 0)
#Divise les données en deux ensemble de la matrice_spam et de la colonne spam du tableau df, une partie entrainement à 70% et test à 30%

In [15]:
matrice_spam.shape #Affiche la taille de matrice_spam, 5695 lignes et 37229 mots uniques

(5695, 37229)

In [16]:
from sklearn.naive_bayes import MultinomialNB #Importation de MultinomialNB
classification = MultinomialNB().fit(X_train, y_train) #Permets la classification avec des fonctionnalités discrètes comme le nombre d'apparations d'un mot
#Classifie la partie training de nos données

In [17]:
print(classification.predict(X_train)) #Affiche les prédictions de matrice_spam
print(y_train) #Affiche les vrais valeurs
#On ne voit pas toutes les valeurs

[0 0 0 ... 0 0 0]
4893    0
3626    0
4382    0
5525    0
1406    0
       ..
4950    0
3273    0
1653    0
2611    0
2736    0
Name: spam, Length: 3986, dtype: int64


In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score #Importation classification-report, confusion_matrix et de accuracy_score
predictions = classification.predict(X_train) #Predictions sur les données d'entrainements de matrice_spam 80% de données
print('Matrice de confusion : \n', confusion_matrix(y_train, predictions))
#Fais la matrice de confusion entre les vrais valeur (df['spam']) et les prédictions
print()
print()
print('Précision: ', accuracy_score(y_train, predictions))
#Affiche la précision de notre modèle qui compare entre les vrais valeurs (df['spam']) et les prédictions

Matrice de confusion : 
 [[3011   11]
 [   0  964]]


Précision:  0.9972403411941796


In [19]:
#Même test avec les valeur de test 30% de données
print(classification.predict(X_test))
print(y_test)

[1 0 0 ... 0 0 1]
977     1
3275    0
4163    0
751     1
3244    0
       ..
4208    0
2450    0
4509    0
4657    0
3037    0
Name: spam, Length: 1709, dtype: int64


In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
predictions = classification.predict(X_test)
print('Matrice de confusion : \n', confusion_matrix(y_test, predictions))
print()
print()
print('Précision: ', accuracy_score(y_test, predictions))

Matrice de confusion : 
 [[1292   13]
 [   4  400]]


Précision:  0.9900526623756583
